# W_GL_BALANCE_F ETL Process
### Staging to Fact - Dimension Lookups and MERGE

In [ ]:
%sql
-- 1) PARAMETERS: Derive ETL control parameters
CREATE OR REPLACE TEMP VIEW etl_params AS
SELECT
  CASE WHEN COUNT(*) > 0 THEN 'Y' ELSE 'N' END AS IS_INCREMENTAL,
  COALESCE(date_format(MIN(LAST_MAX_DATE), 'yyyy-MM-dd HH:mm:ss'), '1970-01-01 00:00:00') AS LAST_EXTRACT_DATE,
  380 AS DATASOURCE_NUM_ID, '__NOT_APPLICABLE__' AS ETL_USAGE_CODE, '1' AS TENANT_ID,
  2623946 AS ETL_PROC_WID, 12345 AS EXECUTION_ID, 30 AS PRUNE_DAYS
FROM workspace.oracle_edw.w_etl_load_dates
WHERE PACKAGE_NAME = 'SILOS_SIL_GLBALANCEFACT'
  AND DATASOURCE_NUM_ID = 380 AND ETL_USAGE_CODE = '__NOT_APPLICABLE__' AND COMMITTED = '1';

In [ ]:
%sql
-- 2) STAGING WITH DIMENSION LOOKUPS: Join staging to dimension tables
CREATE OR REPLACE TEMP VIEW stg_with_dimensions AS
SELECT
  fs.gl_account_id, fs.balance_date, fs.db_cr_ind,
  fs.balance_acct_amt, fs.balance_loc_amt, fs.activity_acct_amt, fs.activity_loc_amt,
  fs.acct_curr_code, fs.loc_curr_code,
  fs.adjustment_flag, fs.balance_type_flag, fs.translated_flag, fs.summary_account_flag,
  fs.last_update_date, fs.aux1_changed_on_dt, fs.aux2_changed_on_dt,
  fs.financial_gl_flg, fs.budgetary_control_flg,
  fs.carry_forward_acct_amt, fs.carry_forward_loc_amt,
  fs.integration_id, fs.datasource_num_id, fs.tenant_id, fs.x_custom,
  -- Dimension WID lookups
  COALESCE(d_account.row_wid, 0) AS gl_account_wid,
  COALESCE(d_period.row_wid, 0) AS period_wid,
  COALESCE(d_budget_period.row_wid, 0) AS budget_period_wid,
  COALESCE(d_ledger.row_wid, 0) AS ledger_wid,
  COALESCE(d_budget_ledger.row_wid, 0) AS budget_ledger_wid,
  COALESCE(d_xact_type.row_wid, 0) AS xact_type_wid,
  COALESCE(d_budget.row_wid, 0) AS budget_wid,
  COALESCE(d_busn_area.row_wid, 0) AS busn_area_org_wid,
  COALESCE(d_mcal_context.mcal_day_wid, 0) AS balance_day_wid,
  -- Derived fields
  fs.ledger_id AS balancing_segment_id,
  fs.integration_id AS balance_id
FROM workspace.oracle_edw.w_gl_balance_fs fs
LEFT JOIN workspace.oracle_edw.w_gl_account_d d_account
  ON fs.gl_account_id = d_account.integration_id
  AND fs.datasource_num_id = d_account.datasource_num_id
  AND d_account.delete_flg = 'N'
LEFT JOIN workspace.oracle_edw.w_mcal_period_d d_period
  ON fs.period_id = d_period.integration_id
  AND fs.datasource_num_id = d_period.datasource_num_id
  AND d_period.delete_flg = 'N'
LEFT JOIN workspace.oracle_edw.w_mcal_period_d d_budget_period
  ON fs.budget_period_id = d_budget_period.integration_id
  AND fs.datasource_num_id = d_budget_period.datasource_num_id
  AND d_budget_period.delete_flg = 'N'
LEFT JOIN workspace.oracle_edw.w_ledger_d d_ledger
  ON fs.ledger_id = d_ledger.integration_id
  AND fs.datasource_num_id = d_ledger.datasource_num_id
  AND d_ledger.delete_flg = 'N'
LEFT JOIN workspace.oracle_edw.w_ledger_d d_budget_ledger
  ON fs.budget_ledger_id = d_budget_ledger.integration_id
  AND fs.datasource_num_id = d_budget_ledger.datasource_num_id
  AND d_budget_ledger.delete_flg = 'N'
LEFT JOIN workspace.oracle_edw.w_xact_type_d d_xact_type
  ON fs.balance_type_id = d_xact_type.integration_id
  AND fs.datasource_num_id = d_xact_type.datasource_num_id
  AND d_xact_type.delete_flg = 'N'
LEFT JOIN workspace.oracle_edw.w_budget_d d_budget
  ON fs.budget_id = d_budget.integration_id
  AND fs.datasource_num_id = d_budget.datasource_num_id
  AND d_budget.delete_flg = 'N'
LEFT JOIN workspace.oracle_edw.w_int_org_d d_busn_area
  ON fs.busn_area_org_id = d_busn_area.integration_id
  AND fs.datasource_num_id = d_busn_area.datasource_num_id
  AND fs.balance_date >= d_busn_area.effective_from_dt
  AND fs.balance_date < d_busn_area.effective_to_dt
  AND d_busn_area.delete_flg = 'N'
LEFT JOIN workspace.oracle_edw.w_mcal_context_g d_mcal_context
  ON fs.ledger_id = d_mcal_context.ledger_id
  AND fs.datasource_num_id = d_mcal_context.datasource_num_id
  AND d_mcal_context.delete_flg = 'N';

In [ ]:
%sql
-- 3) MERGE: Insert new and update existing fact records
MERGE INTO workspace.oracle_edw.w_gl_balance_f AS tgt
USING stg_with_dimensions AS src
ON tgt.datasource_num_id = src.datasource_num_id
  AND tgt.integration_id = src.integration_id
WHEN MATCHED THEN UPDATE SET
  tgt.gl_account_wid = src.gl_account_wid,
  tgt.balance_date = src.balance_date,
  tgt.balance_day_wid = src.balance_day_wid,
  tgt.db_cr_ind = src.db_cr_ind,
  tgt.balance_acct_amt = src.balance_acct_amt,
  tgt.balance_loc_amt = src.balance_loc_amt,
  tgt.activity_acct_amt = src.activity_acct_amt,
  tgt.activity_loc_amt = src.activity_loc_amt,
  tgt.acct_curr_code = src.acct_curr_code,
  tgt.loc_curr_code = src.loc_curr_code,
  tgt.period_wid = src.period_wid,
  tgt.budget_period_wid = src.budget_period_wid,
  tgt.ledger_wid = src.ledger_wid,
  tgt.budget_ledger_wid = src.budget_ledger_wid,
  tgt.xact_type_wid = src.xact_type_wid,
  tgt.budget_wid = src.budget_wid,
  tgt.busn_area_org_wid = src.busn_area_org_wid,
  tgt.adjustment_flag = src.adjustment_flag,
  tgt.balance_type_flag = src.balance_type_flag,
  tgt.translated_flag = src.translated_flag,
  tgt.summary_account_flag = src.summary_account_flag,
  tgt.financial_gl_flg = src.financial_gl_flg,
  tgt.budgetary_control_flg = src.budgetary_control_flg,
  tgt.carry_forward_acct_amt = src.carry_forward_acct_amt,
  tgt.carry_forward_loc_amt = src.carry_forward_loc_amt,
  tgt.balancing_segment_id = src.balancing_segment_id,
  tgt.balance_id = src.balance_id,
  tgt.last_update_date = src.last_update_date,
  tgt.aux1_changed_on_dt = src.aux1_changed_on_dt,
  tgt.aux2_changed_on_dt = src.aux2_changed_on_dt,
  tgt.tenant_id = src.tenant_id,
  tgt.x_custom = src.x_custom,
  tgt.w_update_dt = current_timestamp(),
  tgt.etl_proc_wid = (SELECT ETL_PROC_WID FROM etl_params)
WHEN NOT MATCHED THEN INSERT (
  gl_account_wid, balance_date, balance_day_wid, db_cr_ind,
  balance_acct_amt, balance_loc_amt, activity_acct_amt, activity_loc_amt,
  acct_curr_code, loc_curr_code,
  period_wid, budget_period_wid, ledger_wid, budget_ledger_wid,
  xact_type_wid, budget_wid, busn_area_org_wid,
  adjustment_flag, balance_type_flag, translated_flag, summary_account_flag,
  financial_gl_flg, budgetary_control_flg,
  carry_forward_acct_amt, carry_forward_loc_amt,
  balancing_segment_id, balance_id,
  last_update_date, aux1_changed_on_dt, aux2_changed_on_dt,
  datasource_num_id, integration_id, tenant_id, x_custom,
  w_insert_dt, w_update_dt, etl_proc_wid
) VALUES (
  src.gl_account_wid, src.balance_date, src.balance_day_wid, src.db_cr_ind,
  src.balance_acct_amt, src.balance_loc_amt, src.activity_acct_amt, src.activity_loc_amt,
  src.acct_curr_code, src.loc_curr_code,
  src.period_wid, src.budget_period_wid, src.ledger_wid, src.budget_ledger_wid,
  src.xact_type_wid, src.budget_wid, src.busn_area_org_wid,
  src.adjustment_flag, src.balance_type_flag, src.translated_flag, src.summary_account_flag,
  src.financial_gl_flg, src.budgetary_control_flg,
  src.carry_forward_acct_amt, src.carry_forward_loc_amt,
  src.balancing_segment_id, src.balance_id,
  src.last_update_date, src.aux1_changed_on_dt, src.aux2_changed_on_dt,
  src.datasource_num_id, src.integration_id, src.tenant_id, src.x_custom,
  current_timestamp(), current_timestamp(), (SELECT ETL_PROC_WID FROM etl_params)
);

In [ ]:
%sql
-- 4) UPDATE CONTROL TABLE
MERGE INTO workspace.oracle_edw.w_etl_load_dates AS tgt
USING (
  SELECT p.DATASOURCE_NUM_ID, 'SILOS_SIL_GLBALANCEFACT' AS package_name,
    'W_GL_BALANCE_F' AS target_table_name, p.ETL_USAGE_CODE, p.ETL_PROC_WID, p.EXECUTION_ID AS load_plan_id,
    current_timestamp() AS etl_load_date, CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS committed,
    date_add(current_timestamp(), -p.PRUNE_DAYS) AS wip_load_start_date FROM etl_params p
) src ON tgt.datasource_num_id = src.datasource_num_id AND tgt.package_name = src.package_name
WHEN MATCHED THEN UPDATE SET tgt.target_table_name = src.target_table_name, tgt.etl_load_date = src.etl_load_date, tgt.committed = src.committed
WHEN NOT MATCHED THEN INSERT VALUES (src.datasource_num_id, src.package_name, src.target_table_name, src.etl_usage_code,
  src.etl_proc_wid, src.load_plan_id, src.wip_load_start_date, NULL, src.etl_load_date, src.committed);

In [ ]:
%sql
-- 5) INSERT HISTORY LOG
INSERT INTO workspace.oracle_edw.w_etl_load_dates_log
SELECT p.DATASOURCE_NUM_ID, 'SILOS_SIL_GLBALANCEFACT', 'W_GL_BALANCE_F', p.ETL_USAGE_CODE,
  p.ETL_PROC_WID, p.EXECUTION_ID, p.ETL_PROC_WID, date_add(current_timestamp(), -p.PRUNE_DAYS), NULL,
  current_timestamp(), CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END FROM etl_params p;

In [ ]:
%sql
-- 6) CLEANUP
DROP VIEW IF EXISTS stg_with_dimensions;
DROP VIEW IF EXISTS etl_params;